In [1]:
# Install necessary libraries
!pip install firebase-admin yfinance pandas matplotlib --quiet

# Import Libraries
import firebase_admin
from firebase_admin import credentials, firestore
import yfinance as yf
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Initialize Firebase
if not firebase_admin._apps:
    firebase_json_file = "/content/drive/MyDrive/Colab Notebooks/firebase-key.json"
    cred = credentials.Certificate(firebase_json_file)
    firebase_admin.initialize_app(cred)
    print("Firebase initialized successfully!")

# Initialize Firestore client
db = firestore.client()
print("Firestore client connected!")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Firebase initialized successfully!
Firestore client connected!


In [2]:
# Define Tickers for the Mag7 Index
tickers = ['BTC-USD', 'GOOGL', 'MSFT', 'AMZN', 'AAPL', 'META', 'NVDA', 'TSLA']

# Fetch historical financial data
print("Downloading financial data...")
data = yf.download(tickers, start="2010-01-01", end=datetime.today().strftime('%Y-%m-%d'))
close_data = data['Close'].reset_index()

print("Data downloaded successfully!")
close_data.head()


[*********************100%***********************]  8 of 8 completed

Data downloaded successfully!


Ticker,Date,AAPL,AMZN,BTC-USD,GOOGL,META,MSFT,NVDA,TSLA
0,2010-01-04,7.643214,6.6950,NaN,15.684434,NaN,30.950001,0.46225,NaN
1,2010-01-05,7.656429,6.7345,NaN,15.615365,NaN,30.959999,0.46900,NaN
2,2010-01-06,7.534643,6.6125,NaN,15.221722,NaN,30.770000,0.47200,NaN
3,2010-01-07,7.520714,6.5000,NaN,14.867367,NaN,30.450001,0.46275,NaN
4,2010-01-08,7.570714,6.6760,NaN,15.065566,NaN,30.660000,0.46375,NaN


In [3]:
# Upload data to Firestore
print("Uploading data to Firestore...")

for i, row in close_data.iterrows():
    date_str = row['Date'].strftime('%Y-%m-%d')

    # Reference for Firestore Document
    doc_ref = db.collection("Indices").document("BTC_Mag7_Index").collection("DailyData").document(date_str)

    # Data to be uploaded
    daily_data = {ticker: row[ticker] for ticker in tickers}
    daily_data['Date'] = date_str

    # Upload to Firestore
    doc_ref.set(daily_data)

print("Data upload complete!")


Uploading data to Firestore...
Data upload complete!


In [4]:
!pip install plotly --quiet


In [5]:
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

In [6]:
# Fetch data from Firestore
docs = db.collection("Indices").document("BTC_Mag7_Index").collection("DailyData").stream()

# Convert Firestore data to DataFrame
data = [{**doc.to_dict(), "Date": doc.id} for doc in docs]
df = pd.DataFrame(data).set_index("Date")
df.index = pd.to_datetime(df.index)

# Preview the data
df.head()



,MSFT,TSLA,AAPL,GOOGL,NVDA,META,BTC-USD,AMZN
Date,,,,,,,,
2010-01-04,30.950001,NaN,7.643214,15.684434,0.46225,NaN,NaN,6.6950
2010-01-05,30.959999,NaN,7.656429,15.615365,0.46900,NaN,NaN,6.7345
2010-01-06,30.770000,NaN,7.534643,15.221722,0.47200,NaN,NaN,6.6125
2010-01-07,30.450001,NaN,7.520714,14.867367,0.46275,NaN,NaN,6.5000
2010-01-08,30.660000,NaN,7.570714,15.065566,0.46375,NaN,NaN,6.6760


In [7]:
# Normalize Prices
normalized_df = df / df.iloc[0]

# Calculate the BTC/Mag7 Index
normalized_df['BTC_Mag7_Index'] = normalized_df.mean(axis=1)

# Calculate the 150-Day Moving Average
normalized_df['150_MA'] = normalized_df['BTC_Mag7_Index'].rolling(window=150).mean()


In [8]:
# Check the 150-Day Moving Average column for missing values
print(normalized_df[['BTC_Mag7_Index', '150_MA']].isna().sum())

# Display the first and last few rows to confirm data availability
print(normalized_df[['BTC_Mag7_Index', '150_MA']].head(200))
print(normalized_df[['BTC_Mag7_Index', '150_MA']].tail())


BTC_Mag7_Index    1164
150_MA            3892
dtype: int64
            BTC_Mag7_Index    150_MA
Date                                
2010-01-04        1.000000       NaN
2010-01-05        1.003630       NaN
2010-01-06        0.991850       NaN
2010-01-07        0.977536       NaN
2010-01-08        0.988419       NaN
...                    ...       ...
2010-10-12        0.965136  0.905538
2010-10-13        0.972041  0.905550
2010-10-14        0.971193  0.905563
2010-10-15        1.019138  0.905959
2010-10-18        1.028326  0.906356

[200 rows x 2 columns]
            BTC_Mag7_Index  150_MA
Date                              
2024-12-14             NaN     NaN
2024-12-15             NaN     NaN
2024-12-16       76.065144     NaN
2024-12-17       75.381958     NaN
2024-12-18       74.082155     NaN


In [9]:
# Ensure the index is sorted and clean
normalized_df = normalized_df.sort_index()

# Recalculate the 150-Day Moving Average
normalized_df['150_MA'] = normalized_df['BTC_Mag7_Index'].rolling(window=150, min_periods=1).mean()

# Print the first rows of the recalculated MA
print(normalized_df[['BTC_Mag7_Index', '150_MA']].head(200))


            BTC_Mag7_Index    150_MA
Date                                
2010-01-04        1.000000  1.000000
2010-01-05        1.003630  1.001815
2010-01-06        0.991850  0.998493
2010-01-07        0.977536  0.993254
2010-01-08        0.988419  0.992287
...                    ...       ...
2010-10-12        0.965136  0.905538
2010-10-13        0.972041  0.905550
2010-10-14        0.971193  0.905563
2010-10-15        1.019138  0.905959
2010-10-18        1.028326  0.906356

[200 rows x 2 columns]


In [12]:
# Define BTC market cycle peaks (tops) and bottoms (lows)
cycle_tops = ['2013-12-04', '2017-12-17', '2021-11-10']  # Dates for BTC tops
cycle_bottoms = ['2015-01-14', '2018-12-15', '2022-06-18']  # Dates for BTC bottoms

# Convert dates to datetime
cycle_tops = pd.to_datetime(cycle_tops)
cycle_bottoms = pd.to_datetime(cycle_bottoms)

# Debugging: Check the data range in normalized_df
print(f"Data range in normalized_df: {normalized_df.index.min()} to {normalized_df.index.max()}")

# Check if all cycle_tops and cycle_bottoms are within this range
for top in cycle_tops:
    if top < normalized_df.index.min() or top > normalized_df.index.max():
        print(f"Top date {top} is out of range!")

for bottom in cycle_bottoms:
    if bottom < normalized_df.index.min() or bottom > normalized_df.index.max():
        print(f"Bottom date {bottom} is out of range!")

# Helper function to find the nearest date in the index
def get_nearest_date(index, target_date):
    """Find the nearest date in a DatetimeIndex."""
    pos = index.get_indexer([target_date], method='nearest')[0]
    return index[pos]

print(f"Data range in normalized_df: {normalized_df.index.min()} to {normalized_df.index.max()}")

for top in cycle_tops:
    if top < normalized_df.index.min() or top > normalized_df.index.max():
        print(f"Top date {top} is out of range!")
    else:
        print(f"Top date {top} is within range.")

for bottom in cycle_bottoms:
    if bottom < normalized_df.index.min() or bottom > normalized_df.index.max():
        print(f"Bottom date {bottom} is out of range!")
    else:
        print(f"Bottom date {bottom} is within range.")

# Debugging for BTC Tops
print("Checking BTC Tops:")
for top in cycle_tops:
    try:
        nearest_date = get_nearest_date(normalized_df.index, top)
        print(f"Top: {top} -> Nearest Date: {nearest_date}")
    except Exception as e:
        print(f"Error finding nearest date for Top {top}: {e}")

# Debugging for BTC Bottoms
print("Checking BTC Bottoms:")
for bottom in cycle_bottoms:
    try:
        nearest_date = get_nearest_date(normalized_df.index, bottom)
        print(f"Bottom: {bottom} -> Nearest Date: {nearest_date}")
    except Exception as e:
        print(f"Error finding nearest date for Bottom {bottom}: {e}")

for date in list(cycle_tops) + list(cycle_bottoms):
    print(f"Date: {date}, Value in DataFrame: {normalized_df.loc[date, 'BTC_Mag7_Index'] if date in normalized_df.index else 'Date not in index'}")

print(f"Missing data in 'BTC_Mag7_Index':")
print(normalized_df['BTC_Mag7_Index'].isna().sum())
print(normalized_df[normalized_df['BTC_Mag7_Index'].isna()])

# Fill NaN values in the BTC/Mag7 Index
normalized_df['BTC_Mag7_Index'] = normalized_df['BTC_Mag7_Index'].interpolate(method='time').fillna(method='bfill').fillna(method='ffill')

import plotly.graph_objects as go  # Ensure you import Plotly

# Create the Plotly Figure
fig = go.Figure()

# Add the BTC/Mag7 Index
fig.add_trace(go.Scatter(
    x=normalized_df.index,
    y=normalized_df['BTC_Mag7_Index'],
    mode='lines',
    name='BTC/Mag7 Index',
    line=dict(color='red', width=2)
))

# Add the 150-Day Moving Average
fig.add_trace(go.Scatter(
    x=normalized_df.index,
    y=normalized_df['150_MA'],
    mode='lines',
    name='150-Day MA',
    line=dict(color='orange', width=3, dash='dot')
))

# Flags to control legend entries
top_legend_added = False
bottom_legend_added = False

# Highlight BTC Tops (Cycle Peaks)
for top in cycle_tops:
    nearest_date = get_nearest_date(normalized_df.index, top)
    value = normalized_df.loc[nearest_date, 'BTC_Mag7_Index']
    if pd.notna(value):  # Only plot if value is not NaN
        fig.add_trace(go.Scatter(
            x=[nearest_date],
            y=[value],
            mode='markers+text',
            showlegend=False,  # Skip legend entry
            marker=dict(color='blue', size=10),
            text=["BTC Top"],  # Directly label in the plot
            textposition="top center",
            hoverinfo="text",  # Only display label on hover
            hovertext=f"BTC Top: {nearest_date.strftime('%Y-%m-%d')}, Value: {value:.2f}"
        ))
        print(f"Plotted BTC Top at {nearest_date} with value {value}")
    else:
        print(f"Skipping BTC Top at {nearest_date} due to NaN value.")

# Highlight BTC Bottoms (Cycle Lows)
for bottom in cycle_bottoms:
    nearest_date = get_nearest_date(normalized_df.index, bottom)
    value = normalized_df.loc[nearest_date, 'BTC_Mag7_Index']
    if pd.notna(value):  # Only plot if value is not NaN
        fig.add_trace(go.Scatter(
            x=[nearest_date],
            y=[value],
            mode='markers+text',
            showlegend=False,  # Skip legend entry
            marker=dict(color='green', size=10),
            text=["BTC Bottom"],  # Directly label in the plot
            textposition="bottom center",
            hoverinfo="text",  # Only display label on hover
            hovertext=f"BTC Bottom: {nearest_date.strftime('%Y-%m-%d')}, Value: {value:.2f}"
        ))
        print(f"Plotted BTC Bottom at {nearest_date} with value {value}")
    else:
        print(f"Skipping BTC Bottom at {nearest_date} due to NaN value.")



# Debugging Legend Content
print("\nDebugging legend content:")
for trace in fig.data:
    print(f"Trace name: {trace.name}, Show Legend: {trace.showlegend}")

# Customize the chart layout
fig.update_layout(
    title="BTC/Mag7 Index with 150-Day Moving Average, BTC Tops and Bottoms",
    xaxis_title="Date",
    yaxis_title="Normalized Index Value",
    template="plotly_dark",
    hovermode="x unified",
    legend=dict(x=0.01, y=0.99, bordercolor="White", borderwidth=1)
)

# Show the chart
fig.show()


Data range in normalized_df: 2010-01-04 00:00:00 to 2024-12-18 00:00:00
Data range in normalized_df: 2010-01-04 00:00:00 to 2024-12-18 00:00:00
Top date 2013-12-04 00:00:00 is within range.
Top date 2017-12-17 00:00:00 is within range.
Top date 2021-11-10 00:00:00 is within range.
Bottom date 2015-01-14 00:00:00 is within range.
Bottom date 2018-12-15 00:00:00 is within range.
Bottom date 2022-06-18 00:00:00 is within range.
Checking BTC Tops:
Top: 2013-12-04 00:00:00 -> Nearest Date: 2013-12-04 00:00:00
Top: 2017-12-17 00:00:00 -> Nearest Date: 2017-12-17 00:00:00
Top: 2021-11-10 00:00:00 -> Nearest Date: 2021-11-10 00:00:00
Checking BTC Bottoms:
Bottom: 2015-01-14 00:00:00 -> Nearest Date: 2015-01-14 00:00:00
Bottom: 2018-12-15 00:00:00 -> Nearest Date: 2018-12-15 00:00:00
Bottom: 2022-06-18 00:00:00 -> Nearest Date: 2022-06-18 00:00:00
Date: 2013-12-04 00:00:00, Value in DataFrame: 1.8664402760432277
Date: 2017-12-17 00:00:00, Value in DataFrame: 6.287374041241437
Date: 2021-11-10 0

<ipython-input-12-6414a60c80e6>:67: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  normalized_df['BTC_Mag7_Index'] = normalized_df['BTC_Mag7_Index'].interpolate(method='time').fillna(method='bfill').fillna(method='ffill')
